In [2]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import ensemble, metrics, model_selection, naive_bayes

import lightgbm as lgb
from tqdm import tqdm

In [2]:
X_train = pd.read_csv('df_train_morph.csv')
X_test = pd.read_csv('df_test_morph.csv')

In [3]:
for df in [X_train, X_test]:
    df.loc[df['Анамнез заболевания'].isnull(), 'Анамнез заболевания'] = "[]"
    df.loc[df['Общее состояние'].isnull(), 'Общее состояние'] = "[]"
    df.loc[df['аллергия'].isnull(), 'аллергия'] = "[]"
    df.loc[df['Внешний осмотр'].isnull(), 'Внешний осмотр'] = "[]"

In [4]:
X_train

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
0,0,J06.0,"['острый', 'ларингофарингит']",29,2,[],[],[],"['на', 'жевательный', 'поверхность', '2.6-', '...",1
1,1,N76.1,"['подострый', 'хронический', 'вагинит']",45,2,[],[],"['считать', 'больной', 'протяжение', 'многий',...",[],1
2,2,L23.9,"['аллергический', 'контактный', 'дерматит,', '...",63,2,[],[],"['больной', 'около', '2-х', 'дней,', 'появитьс...",[],1
3,3,N48.1,['баланопостить'],43,2,[],['отяготить'],[],[],1
4,4,Z00.0,"['общий', 'медицинский', 'осмотр']",29,1,[],[],"['провести', 'полный', 'обследование', 'операт...",[],1
5,5,Z01,"['другой', 'специальный', 'осмотр', 'обследова...",50,2,['удовлетворительное.'],['отяготить'],[],[],1
6,6,M23.2,"['поражение', 'мениск', 'результат', 'старое',...",42,2,[],[],"['проводить', 'лечение,', 'прибыть', 'коррекци...",[],1
7,7,E05.0,"['тиреотоксикоз', 'диффузный', 'зоб']",85,1,[],[],"['вышеописанный', 'жалоба', 'беспокоить', '03....",[],0
8,8,N77.1*,"['вагинит,', 'вульвит', 'вульвовагинит', 'инфе...",28,2,"['общий', 'самочувствие', 'удовлетворительное....",[],[],[],1
9,9,H60,"['наружный', 'отит']",38,1,[],[],"['ранее', 'эндокринолог', 'обращаться', 'обсле...",[],1


In [6]:
Y_train = X_train['Revisit']

In [6]:
# X_train.drop(['Revisit'], axis=1, inplace=True)

In [7]:
def tfidfAnamnes(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['Анамнез заболевания'].values.tolist() + X_test['Анамнез заболевания'].values.tolist())
    
    train_tfidf = tfidf_vec.transform(X_train['Анамнез заболевания'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['Анамнез заболевания'].values.tolist())
    
    return train_tfidf, test_tfidf, full_tfidf


def tfidfDiagnoz(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['Диагноз'].values.tolist() + X_test['Диагноз'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['Диагноз'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['Диагноз'].values.tolist())
    return train_tfidf, test_tfidf, full_tfidf


def tfidfGeneral(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['Общее состояние'].values.tolist() + X_test['Общее состояние'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['Общее состояние'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['Общее состояние'].values.tolist())
    return train_tfidf, test_tfidf, full_tfidf


def tfidfAllergy(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['аллергия'].values.tolist() + X_test['аллергия'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['аллергия'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['аллергия'].values.tolist())
    return train_tfidf, test_tfidf, full_tfidf

def tfidfOutside(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['Внешний осмотр'].values.tolist() + X_test['Внешний осмотр'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['Внешний осмотр'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['Внешний осмотр'].values.tolist())
    return train_tfidf, test_tfidf, full_tfidf
    
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_tfidf_MNB(X_train, X_test, Y_train, func):
    train_tfidf, test_tfidf, full_tfidf = func(X_train, X_test)
    
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 2])
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)  
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y[:, 1].reshape((-1, 1))
        cv_scores.append(metrics.log_loss(val_y.values.reshape((-1, 1)), pred_val_y))
    
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train, pred_full_test

In [8]:
%%time
for f in [tfidfAnamnes, tfidfDiagnoz, tfidfGeneral, tfidfAllergy, tfidfOutside]:
    pred_train, pred_test = do_tfidf_MNB(X_train, X_test, Y_train, f)
    X_train["tfidf_nb_eap_{}".format(f.__name__)] = pred_train[:,0]
    X_test["tfidf_nb_eap_{}".format(f.__name__)] = pred_test[:,0]

Mean cv score :  0.827819813898189
Mean cv score :  0.4834655128678458
Mean cv score :  0.4866706184970001
Mean cv score :  0.4645988299175462
Mean cv score :  0.46333357521506874
CPU times: user 1min 15s, sys: 1.68 s, total: 1min 16s
Wall time: 1min 16s


In [9]:
X_train

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,tfidf_nb_eap_tfidfAnamnes,tfidf_nb_eap_tfidfDiagnoz,tfidf_nb_eap_tfidfGeneral,tfidf_nb_eap_tfidfAllergy,tfidf_nb_eap_tfidfOutside
0,0,J06.0,"['острый', 'ларингофарингит']",29,2,[],[],[],"['на', 'жевательный', 'поверхность', '2.6-', '...",1,0.829234,0.929980,0.829234,0.829234,0.999980
1,1,N76.1,"['подострый', 'хронический', 'вагинит']",45,2,[],[],"['считать', 'больной', 'протяжение', 'многий',...",[],1,0.997441,0.975043,0.828135,0.828135,0.828135
2,2,L23.9,"['аллергический', 'контактный', 'дерматит,', '...",63,2,[],[],"['больной', 'около', '2-х', 'дней,', 'появитьс...",[],1,0.999158,0.947991,0.828864,0.828864,0.828864
3,3,N48.1,['баланопостить'],43,2,[],['отяготить'],[],[],1,0.829234,0.806314,0.829234,0.961749,0.829234
4,4,Z00.0,"['общий', 'медицинский', 'осмотр']",29,1,[],[],"['провести', 'полный', 'обследование', 'операт...",[],1,0.982401,0.596475,0.829234,0.829234,0.829234
5,5,Z01,"['другой', 'специальный', 'осмотр', 'обследова...",50,2,['удовлетворительное.'],['отяготить'],[],[],1,0.828864,0.672406,0.913367,0.960559,0.828864
6,6,M23.2,"['поражение', 'мениск', 'результат', 'старое',...",42,2,[],[],"['проводить', 'лечение,', 'прибыть', 'коррекци...",[],1,0.994917,0.149013,0.828135,0.828135,0.828135
7,7,E05.0,"['тиреотоксикоз', 'диффузный', 'зоб']",85,1,[],[],"['вышеописанный', 'жалоба', 'беспокоить', '03....",[],0,0.978885,0.873505,0.829234,0.829234,0.829234
8,8,N77.1*,"['вагинит,', 'вульвит', 'вульвовагинит', 'инфе...",28,2,"['общий', 'самочувствие', 'удовлетворительное....",[],[],[],1,0.828135,0.999757,0.999581,0.828135,0.828135
9,9,H60,"['наружный', 'отит']",38,1,[],[],"['ранее', 'эндокринолог', 'обращаться', 'обсле...",[],1,0.993412,0.778545,0.828727,0.828727,0.828727


In [10]:
def tfidfChars(X_train, X_test, field):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,5),analyzer='char')
    full_tfidf = tfidf_vec.fit_transform(X_train[field].values.tolist() + X_test[field].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train[field].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test[field].values.tolist())
    return train_tfidf, test_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do(X_train, X_test, Y_train, field):
    train_tfidf,test_tfidf = tfidfChars(X_train,X_test, field)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017) 
    for dev_index, val_index in kf.split(X_train): 
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

In [11]:
%%time
for f in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']:
    pred_train, pred_test = do(X_train,X_test,Y_train, f)
    X_train["tfidf_char_nb_eap_{}".format(f)] = pred_train[:,0]
    X_test["tfidf_char_nb_eap_{}".format(f)] = pred_test[:,0]

Mean cv score :  0.7800425639125961
Mean cv score :  0.546997423899825
Mean cv score :  0.5210017219423105
Mean cv score :  1.2012516743091965
Mean cv score :  0.480461203111453
CPU times: user 7min 56s, sys: 13.3 s, total: 8min 9s
Wall time: 8min 9s


In [12]:
X_train.shape, X_test.shape

((283086, 20), (100000, 20))

In [13]:
X_train.to_csv('df_train_stat_small.csv')
X_test.to_csv('df_test_stat_small.csv')

In [14]:
def countWords(X_train,X_test, field):
    count_vec = CountVectorizer(ngram_range=(1,3))
    count_vec.fit(X_train[field].values.tolist() + X_test[field].values.tolist())
    train_count = count_vec.transform(X_train[field].values.tolist())
    test_count = count_vec.transform(X_test[field].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_MNB(X_train,X_test,Y_train, field):
    train_count,test_count=countWords(X_train,X_test, field)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

In [15]:
%%time
for f in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']:
    pred_train, pred_test = do_count_MNB(X_train,X_test,Y_train, f)
    X_train["count_words_nb_eap_{}".format(f)] = pred_train[:,0]
    X_test["count_words_nb_eap_{}".format(f)] = pred_test[:,0]

Mean cv score :  1.0056071455333364
Mean cv score :  0.6302613018423485
Mean cv score :  0.5122894263872634
Mean cv score :  2.2787687295761567
Mean cv score :  0.5006828915306023
CPU times: user 1min 12s, sys: 1.41 s, total: 1min 13s
Wall time: 1min 13s


In [16]:
X_train.shape

(283086, 25)

In [19]:
X_train.to_csv('df_train_stat.csv')
X_test.to_csv('df_test_stat.csv')

In [4]:
def countChars(X_train,X_test, f):
    count_vec = CountVectorizer(ngram_range=(1,7),analyzer='char')
    count_vec.fit(X_train[f].values.tolist() + X_test[f].values.tolist())
    train_count = count_vec.transform(X_train[f].values.tolist())
    test_count = count_vec.transform(X_test[f].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_chars_MNB(X_train,X_test,Y_train, f):
    train_count,test_count=countChars(X_train,X_test, f)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 2])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

In [7]:
%%time
for f in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']:
    pred_train, pred_test = do_count_chars_MNB(X_train,X_test,Y_train, f)
    X_train["count_chars_nb_eap_{}".format(f)] = pred_train[:,0]
    X_test["count_chars_nb_eap_{}".format(f)] = pred_test[:,0]

Mean cv score :  9.897131478983784
Mean cv score :  1.4712417117997993
Mean cv score :  1.1559236601658278
Mean cv score :  4.2050674284020735
Mean cv score :  2.0887756932402697
CPU times: user 12min 55s, sys: 20.3 s, total: 13min 16s
Wall time: 13min 15s


In [8]:
X_train

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,...,count_words_nb_eap_аллергия,count_words_nb_eap_Анамнез заболевания,count_words_nb_eap_Внешний осмотр,Код_диагноза_0,Код_диагноза_1,count_chars_nb_eap_Диагноз,count_chars_nb_eap_Общее состояние,count_chars_nb_eap_аллергия,count_chars_nb_eap_Анамнез заболевания,count_chars_nb_eap_Внешний осмотр
0,0,1327,"['острый', 'ларингофарингит']",29,2,[],[],[],"['на', 'жевательный', 'поверхность', '2.6-', '...",1,...,0.170766,1.707658e-01,1.884889e-15,10,6,3.445999e-19,7.724011e-01,6.081228e-01,2.984199e-02,1.115719e-296
1,1,2469,"['подострый', 'хронический', 'вагинит']",45,2,[],[],"['считать', 'больной', 'протяжение', 'многий',...",[],1,...,0.171865,3.496850e-17,1.718646e-01,14,76,1.136331e-34,7.641650e-01,6.247051e-01,5.274887e-33,9.284846e-01
2,2,1701,"['аллергический', 'контактный', 'дерматит,', '...",63,2,[],[],"['больной', 'около', '2-х', 'дней,', 'появитьс...",[],1,...,0.171136,5.596109e-09,1.711360e-01,12,23,2.553572e-06,7.461672e-01,6.186476e-01,2.657331e-17,9.109264e-01
3,3,2408,['баланопостить'],43,2,[],['отяготить'],[],[],1,...,0.063228,1.707658e-01,1.707658e-01,14,48,9.999924e-01,7.724011e-01,5.889044e-11,2.984199e-02,9.151126e-01
4,4,3324,"['общий', 'медицинский', 'осмотр']",29,1,[],[],"['провести', 'полный', 'обследование', 'операт...",[],1,...,0.170766,4.949576e-04,1.707658e-01,25,0,1.000000e+00,7.724011e-01,6.081228e-01,3.537675e-08,9.151126e-01
5,5,3330,"['другой', 'специальный', 'осмотр', 'обследова...",50,2,['удовлетворительное.'],['отяготить'],[],[],1,...,0.065309,1.711360e-01,1.711360e-01,25,1,1.000000e+00,1.843164e-03,4.745245e-10,3.078752e-02,9.109264e-01
6,6,1987,"['поражение', 'мениск', 'результат', 'старое',...",42,2,[],[],"['проводить', 'лечение,', 'прибыть', 'коррекци...",[],1,...,0.171865,1.444603e-04,1.718646e-01,13,23,1.000000e+00,7.641650e-01,6.247051e-01,1.129032e-09,9.284846e-01
7,7,446,"['тиреотоксикоз', 'диффузный', 'зоб']",85,1,[],[],"['вышеописанный', 'жалоба', 'беспокоить', '03....",[],0,...,0.170766,3.725213e-03,1.707658e-01,5,5,7.952606e-17,7.724011e-01,6.081228e-01,1.326836e-22,9.151126e-01
8,8,2476,"['вагинит,', 'вульвит', 'вульвовагинит', 'инфе...",28,2,"['общий', 'самочувствие', 'удовлетворительное....",[],[],[],1,...,0.171865,1.718646e-01,1.718646e-01,14,77,2.556508e-231,8.735637e-43,6.247051e-01,2.996486e-02,9.284846e-01
9,9,996,"['наружный', 'отит']",38,1,[],[],"['ранее', 'эндокринолог', 'обращаться', 'обсле...",[],1,...,0.171273,1.294402e-03,1.712729e-01,8,60,9.998953e-01,7.585750e-01,6.264281e-01,1.710400e-06,9.311411e-01


In [9]:
X_train.to_csv('df_train_stat_2.csv')
X_test.to_csv('df_test_stat_2.csv')

In [23]:
# X_train.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
X_train = X_train.set_index('ID')

# X_test.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
X_test = X_test.set_index('ID')

In [24]:
X_train.to_csv('df_train_stat.csv')
X_test.to_csv('df_test_stat.csv')

In [28]:
X_train.shape

(283086, 24)

In [29]:
X_train['Код_диагноза_0'] = X_train['Код_диагноза'].apply(lambda x: x[0])
X_test['Код_диагноза_0'] = X_test['Код_диагноза'].apply(lambda x: x[0])

In [30]:
lbl_enc = preprocessing.LabelEncoder()
X_train['Код_диагноза_0'] = lbl_enc.fit_transform(X_train['Код_диагноза_0'])
X_test['Код_диагноза_0'] = lbl_enc.transform(X_test['Код_диагноза_0'])

In [31]:
X_train['Код_диагноза_1'] = X_train['Код_диагноза'].apply(lambda x: x[1:3])
X_test['Код_диагноза_1'] = X_test['Код_диагноза'].apply(lambda x: x[1:3])
lbl_enc = preprocessing.LabelEncoder()
X_train['Код_диагноза_1'] = lbl_enc.fit_transform(X_train['Код_диагноза_1'])
X_test['Код_диагноза_1'] = lbl_enc.transform(X_test['Код_диагноза_1'])

In [32]:
lbl_enc = preprocessing.LabelEncoder()
X_train['Код_диагноза'] = lbl_enc.fit_transform(X_train['Код_диагноза'])
X_test['Код_диагноза'] = lbl_enc.fit_transform(X_test['Код_диагноза'])

In [33]:
X_train.to_csv('train_stat.csv')
X_test.to_csv('test_stat.csv')

In [35]:
X_train.shape

(283086, 26)